# Amazon API Data Extraction Test (OffersV2 Migration)

This notebook tests the **OffersV2** API migration for Amazon product discovery:

## Changes from V1 to V2:
- **Resources**: `OFFERS_*` → `OFFERSV2_*`
- **Response**: `offers` → `offers_v2`
- **Price structure**: `price.amount` → `price.money.amount`
- **New fields**: `is_buy_box_winner`, `deal_details`, `availability`

## Steps:
1. **Check SDK version** - Verify OffersV2 resources are available
2. **`SearchItems`** - Get candidate ASINs with OffersV2
3. **`GetItems`** - Enrich ASINs with detailed OffersV2 data

⚠️ **Deadline**: OffersV1 will stop working after **January 30, 2026**


In [10]:
# Cell 1: Install Dependencies & Imports

import sys
!{sys.executable} -m pip install python-dotenv python-amazon-paapi>=1.2.3 gspread google-auth-oauthlib

import json
import time
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

# Import the specific Amazon PA API components we need
from amazon_paapi.sdk.api.default_api import DefaultApi
from amazon_paapi.sdk.models.partner_type import PartnerType
from amazon_paapi.sdk.models.search_items_request import SearchItemsRequest
from amazon_paapi.sdk.models.search_items_resource import SearchItemsResource
from amazon_paapi.sdk.models.get_items_request import GetItemsRequest
from amazon_paapi.sdk.models.get_items_resource import GetItemsResource
from amazon_paapi.sdk.models.delivery_flag import DeliveryFlag
from amazon_paapi.sdk.rest import ApiException

print("✅ Dependencies and imports are ready.")


✅ Dependencies and imports are ready.


In [8]:
# Cell 2.5: Check available OffersV2 resources in SDK
print("🔍 Checking SDK for OffersV2 support...\n")

# Check SearchItemsResource
print("=== SearchItemsResource ===")
search_v2_resources = [attr for attr in dir(SearchItemsResource) if 'OFFERSV2' in attr.upper()]
search_v1_resources = [attr for attr in dir(SearchItemsResource) if 'OFFERS' in attr.upper() and 'V2' not in attr.upper()]

if search_v2_resources:
    print(f"✅ OffersV2 resources available ({len(search_v2_resources)}):")
    for r in search_v2_resources:
        print(f"   - {r}")
else:
    print("⚠️ No OffersV2 resources found in SearchItemsResource")
    print(f"   Available V1 Offers resources: {search_v1_resources}")

print()

# Check GetItemsResource
print("=== GetItemsResource ===")
get_v2_resources = [attr for attr in dir(GetItemsResource) if 'OFFERSV2' in attr.upper()]
get_v1_resources = [attr for attr in dir(GetItemsResource) if 'OFFERS' in attr.upper() and 'V2' not in attr.upper()]

if get_v2_resources:
    print(f"✅ OffersV2 resources available ({len(get_v2_resources)}):")
    for r in get_v2_resources:
        print(f"   - {r}")
else:
    print("⚠️ No OffersV2 resources found in GetItemsResource")
    print(f"   Available V1 Offers resources: {get_v1_resources}")

print()

# Check SDK version
try:
    import amazon_paapi
    print(f"📦 SDK Version: {amazon_paapi.__version__ if hasattr(amazon_paapi, '__version__') else 'Unknown'}")
except:
    print("📦 SDK Version: Could not determine")

print("\n💡 To enable OffersV2, update SDK: pip install python-amazon-paapi>=1.2.3")


🔍 Checking SDK for OffersV2 support...

=== SearchItemsResource ===
⚠️ No OffersV2 resources found in SearchItemsResource
   Available V1 Offers resources: ['OFFERS_LISTINGS_AVAILABILITY_MAXORDERQUANTITY', 'OFFERS_LISTINGS_AVAILABILITY_MESSAGE', 'OFFERS_LISTINGS_AVAILABILITY_MINORDERQUANTITY', 'OFFERS_LISTINGS_AVAILABILITY_TYPE', 'OFFERS_LISTINGS_CONDITION', 'OFFERS_LISTINGS_CONDITION_CONDITIONNOTE', 'OFFERS_LISTINGS_CONDITION_SUBCONDITION', 'OFFERS_LISTINGS_DELIVERYINFO_ISAMAZONFULFILLED', 'OFFERS_LISTINGS_DELIVERYINFO_ISFREESHIPPINGELIGIBLE', 'OFFERS_LISTINGS_DELIVERYINFO_ISPRIMEELIGIBLE', 'OFFERS_LISTINGS_DELIVERYINFO_SHIPPINGCHARGES', 'OFFERS_LISTINGS_ISBUYBOXWINNER', 'OFFERS_LISTINGS_LOYALTYPOINTS_POINTS', 'OFFERS_LISTINGS_MERCHANTINFO', 'OFFERS_LISTINGS_PRICE', 'OFFERS_LISTINGS_PROGRAMELIGIBILITY_ISPRIMEEXCLUSIVE', 'OFFERS_LISTINGS_PROGRAMELIGIBILITY_ISPRIMEPANTRY', 'OFFERS_LISTINGS_PROMOTIONS', 'OFFERS_LISTINGS_SAVINGBASIS', 'OFFERS_SUMMARIES_HIGHESTPRICE', 'OFFERS_SUMMARIES_LOW

In [9]:
# Cell 2: Configuration and API Client Setup (OffersV2)

from config import conf

# --- PAAPI Credentials & Configuration ---
ACCESS_KEY = conf.amazon.access_key
SECRET_KEY = conf.amazon.secret_key
PARTNER_TAG = conf.amazon.associate_tag
HOST = "webservices.amazon.it"
REGION = "eu-west-1"

# --- API Functions ---

def search_api(browse_node_id, item_page=1, min_price=None, max_price=None, 
               delivery_flags=None, min_reviews_rating=None, min_saving_percent=None):
    """
    Calls the SearchItems API for a given category and page with optional filters.
    Uses OffersV2 resources (new API version).
    
    Args:
        browse_node_id: Browse node ID for the category
        item_page: Page number (default: 1)
        min_price: Minimum price in euros (float, e.g., 10.50)
        max_price: Maximum price in euros (float, e.g., 100.00)
        delivery_flags: List of delivery flags (e.g., ["FulfilledByAmazon"])
        min_reviews_rating: Minimum star rating (int, 1-5, e.g., 4)
        min_saving_percent: Minimum discount percentage (int, e.g., 10 for 10%)
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # OffersV2 resources for SearchItems
    resources = [
        SearchItemsResource.ITEMINFO_TITLE,
        SearchItemsResource.IMAGES_PRIMARY_LARGE,
        SearchItemsResource.IMAGES_VARIANTS_LARGE,
        SearchItemsResource.CUSTOMERREVIEWS_COUNT,
        SearchItemsResource.CUSTOMERREVIEWS_STARRATING,
        SearchItemsResource.BROWSENODEINFO_WEBSITESALESRANK,
        SearchItemsResource.BROWSENODEINFO_BROWSENODES,
    ]
    
    # Try to add OffersV2 resources if available in SDK
    offersv2_resources = [
        'OFFERSV2_SUMMARIES_LOWESTPRICE',
        'OFFERSV2_LISTINGS_PRICE',
        'OFFERSV2_LISTINGS_AVAILABILITY',
        'OFFERSV2_LISTINGS_CONDITION',
        'OFFERSV2_LISTINGS_ISBUYBOXWINNER',
    ]
    for res_name in offersv2_resources:
        if hasattr(SearchItemsResource, res_name):
            resources.append(getattr(SearchItemsResource, res_name))
            print(f"✅ Added SearchItemsResource.{res_name}")
        else:
            # Fallback to V1 if V2 not available
            v1_name = res_name.replace('OFFERSV2', 'OFFERS')
            if hasattr(SearchItemsResource, v1_name):
                resources.append(getattr(SearchItemsResource, v1_name))
                print(f"⚠️ OffersV2 not available, using fallback: {v1_name}")
    
    # Convert price to cents for API
    min_price_cents = int(min_price * 100) if min_price else None
    max_price_cents = int(max_price * 100) if max_price else None
    
    # Handle delivery flags
    delivery_flag_list = []
    if delivery_flags:
        if "FulfilledByAmazon" in delivery_flags:
            delivery_flag_list.append(DeliveryFlag.FULFILLEDBYAMAZON)
    
    try:
        request = SearchItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            browse_node_id=browse_node_id,
            item_page=item_page,
            item_count=10,
            sort_by="Featured",
            resources=resources,
            min_price=min_price_cents,
            max_price=max_price_cents,
            min_reviews_rating=min_reviews_rating,
            min_saving_percent=min_saving_percent,
            delivery_flags=delivery_flag_list if delivery_flag_list else None
        )
        response = api_client.search_items(request)
        return response.to_dict()
    except ApiException as e:
        print(f"API Error on page {item_page} for node {browse_node_id}: {e.reason}")
        return None

def get_items_details(asins):
    """Fetches detailed information for a list of ASINs using the GetItems API with OffersV2."""
    if not asins:
        return []
    
    print(f"\n--- Enriching {len(asins)} items with GetItems API (OffersV2) ---")
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # Build resources list safely - only add resources that exist
    resources = []
    
    # Core resources (non-Offers)
    core_resources = [
        'ITEMINFO_TITLE',
        'IMAGES_PRIMARY_LARGE',
        'IMAGES_VARIANTS_LARGE',
        'CUSTOMERREVIEWS_COUNT',
        'CUSTOMERREVIEWS_STARRATING',
        'ITEMINFO_FEATURES',
        'BROWSENODEINFO_WEBSITESALESRANK'
    ]
    
    # OffersV2 resources (new API) with V1 fallbacks
    offersv2_resources = [
        ('OFFERSV2_LISTINGS_PRICE', 'OFFERS_LISTINGS_PRICE'),
        ('OFFERSV2_LISTINGS_AVAILABILITY', 'OFFERS_LISTINGS_AVAILABILITY'),
        ('OFFERSV2_LISTINGS_CONDITION', 'OFFERS_LISTINGS_CONDITION'),
        ('OFFERSV2_LISTINGS_ISBUYBOXWINNER', None),  # No V1 equivalent
        ('OFFERSV2_LISTINGS_SAVINGBASIS', 'OFFERS_LISTINGS_SAVINGBASIS'),
    ]
    
    # Optional resources - try to add them if they exist
    optional_resources = [
        'ITEMINFO_PRODUCT_INFO',
        'ITEMINFO_TECHNICAL_INFO',
        'ITEMINFO_CONTENT_INFO',
        'ITEMINFO_CLASSIFICATIONS',
        'ITEMINFO_BYLINE_INFO',
        'ITEMINFO_EXTERNAL_IDS',
        'ITEMINFO_MANUFACTURE_INFO',
        'ITEMINFO_TRADE_IN_INFO'
    ]
    
    # Add core resources
    for resource_name in core_resources:
        try:
            resource = getattr(GetItemsResource, resource_name)
            resources.append(resource)
        except AttributeError:
            print(f"⚠️ Warning: Resource {resource_name} not available")
    
    # Add OffersV2 resources (with V1 fallback)
    for v2_name, v1_fallback in offersv2_resources:
        if hasattr(GetItemsResource, v2_name):
            resources.append(getattr(GetItemsResource, v2_name))
            print(f"✅ Using OffersV2: {v2_name}")
        elif v1_fallback and hasattr(GetItemsResource, v1_fallback):
            resources.append(getattr(GetItemsResource, v1_fallback))
            print(f"⚠️ OffersV2 not available, using V1: {v1_fallback}")
    
    # Try to add optional resources
    for resource_name in optional_resources:
        try:
            resource = getattr(GetItemsResource, resource_name)
            resources.append(resource)
        except AttributeError:
            pass  # Silently skip if not available

    asin_chunks = [asins[i:i + 10] for i in range(0, len(asins), 10)]
    enriched_items = []

    for i, chunk in enumerate(asin_chunks):
        print(f"Fetching details for batch {i+1}/{len(asin_chunks)}...")
        request = GetItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            item_ids=chunk,
            resources=resources
        )
        try:
            response = api_client.get_items(request).to_dict()
            if response and response.get("items_result") and response["items_result"].get("items"):
                enriched_items.extend(response["items_result"]["items"])
        except ApiException as e:
            print(f"API Error (GetItems) on batch {i+1}: {e.reason}")
        
        time.sleep(1)

    return enriched_items

def extract_price_from_item(item):
    """
    Extract price and savings from item, supporting both OffersV1 and OffersV2.
    Returns (price, savings) tuple.
    """
    price = "N/A"
    savings = "N/A"
    
    # Try OffersV2 first (new API)
    if item.get("offers_v2"):
        offers_v2 = item["offers_v2"]
        # OffersV2 Listings
        if offers_v2.get("listings") and len(offers_v2["listings"]) > 0:
            listing = offers_v2["listings"][0]
            if listing.get("price"):
                price_obj = listing["price"]
                # OffersV2 uses nested "money" structure
                if price_obj.get("money"):
                    price = price_obj["money"].get("display_amount", "N/A")
                else:
                    price = price_obj.get("display_amount", "N/A")
                # Savings
                if price_obj.get("savings"):
                    sav = price_obj["savings"]
                    if sav.get("money"):
                        savings = f"{sav['money'].get('display_amount', 'N/A')} ({sav.get('percentage', '')}%)"
                    else:
                        savings = sav.get("display_amount", "N/A")
        # OffersV2 Summaries
        elif offers_v2.get("summaries") and len(offers_v2["summaries"]) > 0:
            summary = offers_v2["summaries"][0]
            if summary.get("lowest_price"):
                lp = summary["lowest_price"]
                if lp.get("money"):
                    price = lp["money"].get("display_amount", "N/A")
                else:
                    price = lp.get("display_amount", "N/A")
    
    # Fallback to OffersV1 (old API)
    elif item.get("offers"):
        offers = item["offers"]
        # V1 Listings
        if offers.get("listings") and len(offers["listings"]) > 0:
            listing = offers["listings"][0]
            if listing.get("price"):
                price = listing["price"].get("display_amount", "N/A")
                if listing["price"].get("savings"):
                    savings = listing["price"]["savings"].get("display_amount", "N/A")
        # V1 Summaries
        elif offers.get("summaries") and len(offers["summaries"]) > 0:
            summary = offers["summaries"][0]
            if summary.get("lowest_price"):
                price = summary["lowest_price"].get("display_amount", "N/A")
            if summary.get("saving"):
                savings = summary["saving"].get("display_amount", "N/A")
    
    return price, savings


def print_search_results(search_response):
    """Pretty print the search results from SearchItems API (supports OffersV1 & V2)."""
    if not search_response:
        print("❌ No response received from API.")
        return
    
    if "search_result" not in search_response:
        print("❌ No search_result in response.")
        print(f"Response keys: {list(search_response.keys())}")
        return
    
    search_result = search_response["search_result"]
    
    total_results = search_result.get("total_result_count", "Unknown")
    search_index = search_result.get("search_index", "Unknown")
    browse_node_id = search_result.get("browse_node", {}).get("id", "Unknown") if search_result.get("browse_node") else "Unknown"
    browse_node_name = search_result.get("browse_node", {}).get("display_name", "Unknown") if search_result.get("browse_node") else "Unknown"
    
    print(f"\n{'='*80}")
    print(f"📊 Search Results Summary (OffersV2 Compatible)")
    print(f"{'='*80}")
    print(f"Total Results: {total_results}")
    print(f"Search Index: {search_index}")
    print(f"Browse Node ID: {browse_node_id}")
    print(f"Browse Node Name: {browse_node_name}")
    print(f"{'='*80}\n")
    
    items = search_result.get("items", [])
    if not items:
        print("⚠️  No items found in search results.")
        return
    
    print(f"Found {len(items)} items on this page:\n")
    
    for idx, item in enumerate(items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
        
        # Use universal price extraction
        price, savings = extract_price_from_item(item)
        
        image_url = "N/A"
        if item.get("images") and item["images"].get("primary") and item["images"]["primary"].get("large"):
            image_url = item["images"]["primary"]["large"].get("url", "N/A")
        
        review_count = "N/A"
        star_rating = "N/A"
        if item.get("customer_reviews"):
            review_count = item["customer_reviews"].get("count", "N/A")
            if item["customer_reviews"].get("star_rating"):
                star_rating = item["customer_reviews"]["star_rating"].get("display_value", "N/A")
        
        sales_rank = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("website_sales_rank"):
            sales_rank_obj = item["browse_node_info"]["website_sales_rank"]
            if isinstance(sales_rank_obj, dict):
                sales_rank = sales_rank_obj.get("sales_rank", "N/A")
        
        browse_nodes = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("browse_nodes"):
            browse_nodes_list = item["browse_node_info"]["browse_nodes"]
            if browse_nodes_list:
                browse_nodes = browse_nodes_list[0].get("display_name", "N/A")
        
        detail_url = item.get("detail_page_url", "N/A")
        
        print(f"{'─'*80}")
        print(f"Item #{idx}")
        print(f"{'─'*80}")
        print(f"ASIN:         {asin}")
        print(f"Title:        {title}")
        print(f"Price:        {price}")
        if savings != "N/A":
            print(f"Savings:      {savings}")
        print(f"Reviews:      {review_count} ({star_rating} stars)")
        print(f"Sales Rank:   {sales_rank}")
        print(f"Browse Node:  {browse_nodes}")
        if len(image_url) > 80:
            print(f"Image URL:    {image_url[:77]}...")
        else:
            print(f"Image URL:    {image_url}")
        if len(detail_url) > 80:
            print(f"Detail URL:   {detail_url[:77]}...")
        else:
            print(f"Detail URL:   {detail_url}")
        print()
    
    print(f"{'='*80}\n")

def print_enriched_items(enriched_items):
    """Pretty print the enriched items from GetItems API (supports OffersV1 & V2)."""
    if not enriched_items:
        print("❌ No enriched items to display.")
        return
    
    print(f"\n{'='*80}")
    print(f"📦 Enriched Items Details (OffersV2 Compatible)")
    print(f"{'='*80}")
    print(f"Total Items: {len(enriched_items)}\n")
    
    for idx, item in enumerate(enriched_items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
        
        # Use universal price extraction (supports both V1 and V2)
        price, savings = extract_price_from_item(item)
        
        # Check for OffersV2-specific fields
        is_buy_box_winner = None
        if item.get("offers_v2") and item["offers_v2"].get("listings"):
            listing = item["offers_v2"]["listings"][0]
            is_buy_box_winner = listing.get("is_buy_box_winner")
        
        image_url = "N/A"
        if item.get("images") and item["images"].get("primary") and item["images"]["primary"].get("large"):
            image_url = item["images"]["primary"]["large"].get("url", "N/A")
        
        review_count = "N/A"
        star_rating = "N/A"
        if item.get("customer_reviews"):
            review_count = item["customer_reviews"].get("count", "N/A")
            if item["customer_reviews"].get("star_rating"):
                star_rating = item["customer_reviews"]["star_rating"].get("display_value", "N/A")
        
        sales_rank = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("website_sales_rank"):
            sales_rank_obj = item["browse_node_info"]["website_sales_rank"]
            if isinstance(sales_rank_obj, dict):
                sales_rank = sales_rank_obj.get("sales_rank", "N/A")
        
        detail_url = item.get("detail_page_url", "N/A")
        
        # Features (bullet points) - safely get
        features = []
        try:
            features = item.get("item_info", {}).get("features", {}).get("display_values", [])
        except (AttributeError, KeyError, TypeError):
            pass
        
        # Product Info - safely get
        product_info = {}
        try:
            product_info = item.get("item_info", {}).get("product_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        color = product_info.get("color", {}).get("display_value", None) if product_info and product_info.get("color") else None
        size = product_info.get("size", {}).get("display_value", None) if product_info and product_info.get("size") else None
        unit_count = product_info.get("unit_count", {}).get("display_value", None) if product_info and product_info.get("unit_count") else None
        
        # Classifications - safely get
        classifications = {}
        try:
            classifications = item.get("item_info", {}).get("classifications", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        product_group = classifications.get("product_group", {}).get("display_value", None) if classifications and classifications.get("product_group") else None
        binding = classifications.get("binding", {}).get("display_value", None) if classifications and classifications.get("binding") else None
        
        # Content Info - safely get
        content_info = {}
        try:
            content_info = item.get("item_info", {}).get("content_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        pages_count = content_info.get("pages_count", {}).get("display_value", None) if content_info and content_info.get("pages_count") else None
        publication_date = content_info.get("publication_date", {}).get("display_value", None) if content_info and content_info.get("publication_date") else None
        
        # Technical Info - safely get
        technical_info = {}
        try:
            technical_info = item.get("item_info", {}).get("technical_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        technical_info_str = "Available" if technical_info else None
        
        print(f"{'─'*80}")
        print(f"Item #{idx}")
        print(f"{'─'*80}")
        print(f"ASIN:         {asin}")
        print(f"Title:        {title}")
        print(f"Price:        {price}")
        if savings != "N/A":
            print(f"Savings:      {savings}")
        if is_buy_box_winner is not None:
            print(f"Buy Box:      {'✅ Winner' if is_buy_box_winner else '❌ Not winner'}")
        print(f"Reviews:      {review_count} ({star_rating} stars)")
        print(f"Sales Rank:   {sales_rank}")
        if product_group:
            print(f"Product Group: {product_group}")
        if binding:
            print(f"Binding:      {binding}")
        if color:
            print(f"Color:        {color}")
        if size:
            print(f"Size:         {size}")
        if unit_count:
            print(f"Unit Count:   {unit_count}")
        if pages_count:
            print(f"Pages:        {pages_count}")
        if publication_date:
            print(f"Pub Date:     {publication_date}")
        if technical_info_str:
            print(f"Technical Info: {technical_info_str}")
        if features:
            print(f"Features:     {len(features)} bullet points")
            for i, feature in enumerate(features[:3], 1):
                feature_text = feature[:70] + "..." if len(feature) > 70 else feature
                print(f"  {i}. {feature_text}")
            if len(features) > 3:
                print(f"  ... and {len(features) - 3} more")
        print(f"Image URL:    {image_url[:80]}..." if len(image_url) > 80 else f"Image URL:    {image_url}")
        print(f"Detail URL:   {detail_url[:80]}..." if len(detail_url) > 80 else f"Detail URL:   {detail_url}")
        print()
    
    print(f"{'='*80}\n")

print("✅ Configuration and helper functions are defined.")



✅ Configuration and helper functions are defined.


In [10]:
# Cell 4: TEST - Can SDK accept STRING resources instead of ENUMs?
# If this works, we don't need raw HTTP!

print("="*70)
print("🧪 TEST: SDK with STRING resources (not enums)")
print("="*70)

api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)

# Test 1: Try passing strings directly to resources parameter
print("\n📋 Test 1: Pass OffersV2 strings directly to GetItemsRequest...")

try:
    request = GetItemsRequest(
        partner_tag=PARTNER_TAG,
        partner_type=PartnerType.ASSOCIATES,
        marketplace="www.amazon.it",
        item_ids=["B0BP2QZLH7"],
        resources=[
            "ItemInfo.Title",  # String instead of enum!
            "OffersV2.Listings.Price",  # OffersV2 as string!
            "OffersV2.Listings.IsBuyBoxWinner",
        ]
    )
    
    response = api_client.get_items(request)
    result = response.to_dict()
    
    if result and result.get("items_result") and result["items_result"].get("items"):
        item = result["items_result"]["items"][0]
        print(f"✅ SUCCESS! SDK accepted string resources!")
        print(f"   Item keys: {list(item.keys())}")
        
        if "offers_v2" in item:
            print(f"   🎉 OffersV2 data present!")
            listing = item["offers_v2"]["listings"][0]
            price = listing.get("price", {}).get("money", {}).get("display_amount", "N/A")
            print(f"   Price: {price}")
        elif "OffersV2" in item:
            print(f"   🎉 OffersV2 data present (PascalCase)!")
        else:
            print(f"   ⚠️ No OffersV2 in response, only: {list(item.keys())}")
    else:
        print(f"❌ No items in response")
        
except TypeError as e:
    print(f"❌ TypeError: SDK doesn't accept strings - {e}")
except ApiException as e:
    print(f"❌ API Error: {e.reason}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

# Test 2: Try mixing enums and strings
print("\n📋 Test 2: Mix ENUMs and STRING resources...")

try:
    request = GetItemsRequest(
        partner_tag=PARTNER_TAG,
        partner_type=PartnerType.ASSOCIATES,
        marketplace="www.amazon.it",
        item_ids=["B0BP2QZLH7"],
        resources=[
            GetItemsResource.ITEMINFO_TITLE,  # Enum
            "OffersV2.Listings.Price",  # String
        ]
    )
    
    response = api_client.get_items(request)
    print(f"✅ SDK accepts mixed enum/string resources!")
    
except TypeError as e:
    print(f"❌ TypeError: Can't mix - {e}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")


🧪 TEST: SDK with STRING resources (not enums)

📋 Test 1: Pass OffersV2 strings directly to GetItemsRequest...
✅ SUCCESS! SDK accepted string resources!
   Item keys: ['asin', 'browse_node_info', 'customer_reviews', 'detail_page_url', 'images', 'item_info', 'offers', 'parent_asin', 'rental_offers', 'score', 'variation_attributes']
   ⚠️ No OffersV2 in response, only: ['asin', 'browse_node_info', 'customer_reviews', 'detail_page_url', 'images', 'item_info', 'offers', 'parent_asin', 'rental_offers', 'score', 'variation_attributes']

📋 Test 2: Mix ENUMs and STRING resources...
✅ SDK accepts mixed enum/string resources!


In [31]:
# Cell: TEST min_reviews_rating filter in SearchItems API
# Testing if Keywords parameter enables the rating filter

print("="*70)
print("🧪 TEST: min_reviews_rating filter WITH Keywords")
print("="*70)
print("Hypothesis: Adding Keywords makes min_reviews_rating work properly")

# Use a browse node with established products
TEST_BROWSE_NODE = "6311632031"  # Health & Personal Care

def test_rating_filter_with_keywords(browse_node_id, keywords=None, min_rating=None):
    """Test SearchItems with Keywords + min_reviews_rating filter."""
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    resources = [
        SearchItemsResource.ITEMINFO_TITLE,
        SearchItemsResource.CUSTOMERREVIEWS_COUNT,
        SearchItemsResource.CUSTOMERREVIEWS_STARRATING,
        SearchItemsResource.OFFERS_LISTINGS_PRICE,
    ]
    
    try:
        # Build request with optional keywords
        request_params = {
            "partner_tag": PARTNER_TAG,
            "partner_type": PartnerType.ASSOCIATES,
            "marketplace": "www.amazon.it",
            "browse_node_id": browse_node_id,
            "item_page": 1,
            "item_count": 10,
            "sort_by": "Featured",
            "resources": resources,
        }
        
        if keywords:
            request_params["keywords"] = keywords
        if min_rating:
            request_params["min_reviews_rating"] = min_rating
            
        request = SearchItemsRequest(**request_params)
        
        response = api_client.search_items(request)
        result = response.to_dict()
        
        if result and result.get("search_result") and result["search_result"].get("items"):
            items = result["search_result"]["items"]
            label = f"keywords='{keywords}', min_rating={min_rating}"
            print(f"\n📦 Found {len(items)} items ({label}):\n")
            
            for i, item in enumerate(items[:10], 1):
                asin = item.get("asin", "N/A")
                title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")[:40]
                
                rating = "N/A"
                review_count = "N/A"
                if item.get("customer_reviews"):
                    cr = item["customer_reviews"]
                    review_count = cr.get("count", "N/A")
                    if cr.get("star_rating"):
                        rating = cr["star_rating"].get("value", "N/A")
                
                print(f"  {i}. {asin} | ⭐ {rating} | 📝 {review_count} reviews | {title}...")
            
            return items
        else:
            print(f"❌ No items found")
            return []
            
    except ApiException as e:
        print(f"❌ API Error: {e.reason}")
        return []
    except Exception as e:
        print(f"❌ Error: {e}")
        return []

def get_ratings(items):
    """Extract ratings from items."""
    ratings = []
    for item in items:
        if item.get("customer_reviews") and item["customer_reviews"].get("star_rating"):
            rating = item["customer_reviews"]["star_rating"].get("value")
            if rating:
                ratings.append(float(rating))
    return ratings

# TEST 1: BrowseNode only (no keywords, no rating filter)
print("\n" + "-"*70)
print("📋 TEST 1: BrowseNode only (no Keywords, no rating filter)")
print("-"*70)
items_test1 = test_rating_filter_with_keywords(TEST_BROWSE_NODE, keywords=None, min_rating=None)

time.sleep(1.5)

# TEST 2: BrowseNode + Keywords (no rating filter)  
print("\n" + "-"*70)
print("📋 TEST 2: BrowseNode + Keywords='*' (no rating filter)")
print("-"*70)
items_test2 = test_rating_filter_with_keywords(TEST_BROWSE_NODE, keywords="*", min_rating=None)

time.sleep(1.5)

# TEST 3: BrowseNode + Keywords + Rating filter
print("\n" + "-"*70)
print("📋 TEST 3: BrowseNode + Keywords='*' + min_rating=4")
print("-"*70)
items_test3 = test_rating_filter_with_keywords(TEST_BROWSE_NODE, keywords="*", min_rating=4)

time.sleep(1.5)

# TEST 4: Try with actual keyword instead of wildcard
print("\n" + "-"*70)
print("📋 TEST 4: BrowseNode + Keywords='vitamins' + min_rating=4")
print("-"*70)
items_test4 = test_rating_filter_with_keywords(TEST_BROWSE_NODE, keywords="vitamins", min_rating=4)

# ANALYSIS
print("\n" + "="*70)
print("📊 ANALYSIS")
print("="*70)

results = [
    ("TEST 1: Node only", items_test1),
    ("TEST 2: Node + '*'", items_test2),
    ("TEST 3: Node + '*' + rating=4", items_test3),
    ("TEST 4: Node + 'vitamins' + rating=4", items_test4),
]

for name, items in results:
    ratings = get_ratings(items)
    if ratings:
        print(f"\n{name}:")
        print(f"  Items: {len(items)}, With ratings: {len(ratings)}")
        print(f"  Ratings: {ratings}")
        print(f"  Range: {min(ratings):.1f} - {max(ratings):.1f}")
        if "rating=4" in name and min(ratings) >= 4.0:
            print(f"  ✅ Filter WORKS!")
        elif "rating=4" in name:
            print(f"  ⚠️ Filter NOT working (items < 4.0)")
    else:
        print(f"\n{name}: {len(items)} items, 0 with ratings from API")


🧪 TEST: min_reviews_rating filter WITH Keywords
Hypothesis: Adding Keywords makes min_reviews_rating work properly

----------------------------------------------------------------------
📋 TEST 1: BrowseNode only (no Keywords, no rating filter)
----------------------------------------------------------------------

📦 Found 10 items (keywords='None', min_rating=None):

  1. B0F9LF49QD | ⭐ N/A | 📝 N/A reviews | LUCY® 3x Filtri Clarity Pad per Caraffa ...
  2. B0F8HFH8FM | ⭐ N/A | 📝 N/A reviews | Connettore del Filtro dell'Acqua, 1/4" a...
  3. B0F8ZR7SX3 | ⭐ N/A | 📝 N/A reviews | KEEP IT FRESH - Confezione da 10 pezzi R...
  4. B0FQ6RFRNW | ⭐ N/A | 📝 N/A reviews | EWS® | CHIAVE IN PLASTICA PER SERRAGGIO ...
  5. B0FKMVDXNN | ⭐ N/A | 📝 N/A reviews | ZORVYN 10 Pezzi Filtri a Carbone Attivo,...
  6. B0FJWMM8F2 | ⭐ N/A | 📝 N/A reviews | Cucina 84mm Lavello Spina Filtro Acciaio...
  7. B0FGJYCZS6 | ⭐ N/A | 📝 N/A reviews | Idrosal 3 Sacchi di Sale in Pastiglie 10...
  8. B0FGD2S6F7 | ⭐ N/A | 📝

In [11]:
# Example: Get enriched details for ASINs from search results
if 'searched_items' in locals() and searched_items:
    asins = [item.get('asin') for item in searched_items.get('search_result', {}).get('items', []) if item.get('asin')]
    if asins:
        enriched = get_items_details(asins)
        print_enriched_items(enriched)


In [13]:
# Cell 3.8: FINAL TEST - Complete OffersV2 with all 7 valid resources

def get_item_with_offersv2(asin: str):
    """
    Fetch item with ALL valid OffersV2 resources.
    This is the production-ready approach!
    """
    # All 7 valid OffersV2 resources
    offersv2_resources = [
        "OffersV2.Listings.Price",
        "OffersV2.Listings.Availability", 
        "OffersV2.Listings.Condition",
        "OffersV2.Listings.IsBuyBoxWinner",
        "OffersV2.Listings.MerchantInfo",
        "OffersV2.Listings.DealDetails",
        "OffersV2.Listings.LoyaltyPoints",
    ]
    
    # Other useful resources
    other_resources = [
        "ItemInfo.Title",
        "ItemInfo.Features",
        "Images.Primary.Large",
        "Images.Variants.Large",
        "BrowseNodeInfo.WebsiteSalesRank",
    ]
    
    payload = {
        "ItemIds": [asin],
        "PartnerTag": PARTNER_TAG,
        "PartnerType": "Associates",
        "Marketplace": "www.amazon.it",
        "Resources": other_resources + offersv2_resources
    }
    
    payload_json = json.dumps(payload)
    headers = sign_request(HOST, REGION, ACCESS_KEY, SECRET_KEY, payload_json)
    response = requests.post(f"https://{HOST}/paapi5/getitems", headers=headers, data=payload_json)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error: {response.text}")
        return None


def display_offersv2_result(data):
    """Pretty display OffersV2 response."""
    if not data or "ItemsResult" not in data:
        print("No data")
        return
    
    item = data["ItemsResult"]["Items"][0]
    
    print(f"\n{'='*70}")
    print(f"📦 PRODUCT: {item.get('ASIN')}")
    print(f"{'='*70}")
    
    # Title
    title = item.get("ItemInfo", {}).get("Title", {}).get("DisplayValue", "N/A")
    print(f"Title: {title[:60]}...")
    
    # Sales Rank
    if item.get("BrowseNodeInfo", {}).get("WebsiteSalesRank"):
        rank = item["BrowseNodeInfo"]["WebsiteSalesRank"].get("SalesRank", "N/A")
        print(f"Sales Rank: #{rank}")
    
    # OffersV2 Data
    if "OffersV2" not in item:
        print("\n⚠️ No OffersV2 data")
        return
    
    offers = item["OffersV2"]
    print(f"\n{'─'*70}")
    print(f"💰 OFFERSV2 DATA")
    print(f"{'─'*70}")
    
    for i, listing in enumerate(offers.get("Listings", []), 1):
        print(f"\n📋 Listing #{i}")
        
        # Buy Box Winner
        if "IsBuyBoxWinner" in listing:
            print(f"   🏆 Buy Box Winner: {'✅ YES' if listing['IsBuyBoxWinner'] else '❌ NO'}")
        
        # Price (V2 structure: Price.Money.Amount)
        if listing.get("Price"):
            price = listing["Price"]
            if price.get("Money"):
                print(f"   💵 Price: {price['Money'].get('DisplayAmount', 'N/A')}")
            
            # Price per unit
            if price.get("PricePerUnit"):
                print(f"   📏 Per Unit: {price['PricePerUnit'].get('DisplayAmount', 'N/A')}")
            
            # Savings
            if price.get("SavingBasis"):
                basis = price["SavingBasis"]
                if basis.get("Money"):
                    print(f"   💸 Was: {basis['Money'].get('DisplayAmount', 'N/A')}")
                if basis.get("Savings"):
                    sav = basis["Savings"]
                    pct = sav.get("Percentage", "")
                    amt = sav.get("Money", {}).get("DisplayAmount", "")
                    print(f"   🎯 Savings: {amt} ({pct}%)")
        
        # Availability
        if listing.get("Availability"):
            avail = listing["Availability"]
            print(f"   📦 Availability: {avail.get('Message', avail.get('Type', 'N/A'))}")
        
        # Condition
        if listing.get("Condition"):
            cond = listing["Condition"]
            print(f"   ✨ Condition: {cond.get('Value', 'N/A')}")
        
        # Merchant Info
        if listing.get("MerchantInfo"):
            merchant = listing["MerchantInfo"]
            name = merchant.get("Name", "Unknown")
            mid = merchant.get("Id", "N/A")
            print(f"   🏪 Merchant: {name} (ID: {mid[:20]}...)")
        
        # Deal Details (THE KEY NEW FEATURE!)
        if listing.get("DealDetails"):
            deal = listing["DealDetails"]
            print(f"   🔥 DEAL ACTIVE!")
            if deal.get("DealBadge"):
                print(f"      Badge: {deal['DealBadge']}")
            if deal.get("DealPrice"):
                dp = deal["DealPrice"]
                print(f"      Deal Price: {dp.get('Money', {}).get('DisplayAmount', 'N/A')}")
            if deal.get("PercentOff"):
                print(f"      Percent Off: {deal['PercentOff']}%")
            if deal.get("EndTime"):
                print(f"      Ends: {deal['EndTime']}")
        
        # Loyalty Points
        if listing.get("LoyaltyPoints"):
            points = listing["LoyaltyPoints"]
            print(f"   🎁 Loyalty Points: {points.get('Points', 'N/A')}")
    
    # Raw JSON preview
    print(f"\n{'─'*70}")
    print("📄 Raw OffersV2 JSON:")
    print(f"{'─'*70}")
    print(json.dumps(offers, indent=2, ensure_ascii=False)[:1500])


# Run the test
print("="*70)
print("🚀 FINAL OFFERSV2 TEST - ALL 7 RESOURCES")
print("="*70)

result = get_item_with_offersv2("B0BP2QZLH7")
if result:
    display_offersv2_result(result)
    
print("\n\n" + "="*70)
print("✅ CONCLUSION: OffersV2 is READY for production!")
print("="*70)
print("""
Next steps to update the bot:
1. Use raw HTTP requests OR wait for SDK update
2. Request these resources: OffersV2.Listings.*
3. Parse response from item["OffersV2"]["Listings"]
4. Price is now at: listing["Price"]["Money"]["Amount"]
""")


🚀 FINAL OFFERSV2 TEST - ALL 7 RESOURCES


NameError: name 'sign_request' is not defined